# Spectral clustering of network with K-means clustering in fewer dimensions

Jakub Kozieł, Jakub Lis

## Imports

In [1]:
import pandas as pd
import time
import glob
import networkx as nx
import spectral_clustering
import os
import warnings

warnings.filterwarnings("ignore")

## Helper functions

Loading graph from files + reordering clusters.

In [2]:
def load_graph_from_file(path_to_file):
    A = pd.read_csv(path_to_file, header=None)
    num_of_clusters = None
    if 'K=' in path_to_file:
        num_of_clusters = int(path_to_file.split('\\')[-1].split('K=')[-1][:-4])
    return nx.Graph(A), num_of_clusters


def reorder_clusters(clusters):
    change_to = [0] * max(clusters)
    for el in clusters:
        if change_to[el - 1] == 0:
            change_to[el - 1] = max(change_to) + 1
    for i in range(len(clusters)):
        clusters[i] = change_to[clusters[i] - 1]
    return clusters

## Algorithm + writing results

In [3]:
with open('description.txt', 'w') as f:
    f.write("Jakub Kozieł, Jakub Lis\n")
    f.write("https://github.com/liskuba/SNARS_clustering\n")

    for test_file in glob.glob("competition/*.csv"):
        f.write(test_file.split(os.sep)[-1] + ", ")

        G, num_of_clusters = load_graph_from_file(test_file)

        start_time = time.time()

        clusters, num_of_clusters = spectral_clustering.spectral_clustering(G, num_of_clusters)

        f.write(f"{time.time() - start_time}\n")

        result = pd.DataFrame(data={'node': list(G.nodes), 'cluster': reorder_clusters(clusters)})
        result.to_csv(test_file.split(os.sep)[-1], header=False, index=False)